# CAI Lab 6: pLSI with toy corpus


_In this lab session, rather than assigning some task for you to do, I present code that implements the EM algorithm applied to the
toyexample that we saw in class. The main objective of today's lab session is to clarify this model and provide you with code you
can apply to a larger, more realistic corpus in the future._

In [ ]:
import numpy as np
import pandas as pd

pd.set_option('display.precision', 5)
np.set_printoptions(precision = 5)


## 1. The pLSI probabilistic model (using asymmetric formulation)

The central parameters of the model are ($d$ stands for a document, $w$ stands for a word, and $z$ stands for the latent variable indicating topic assignment). 

- $P(z|d)$: the topic proportions for each document ("topic distribution for document $d$"); we have $K N$ such parameters; we use matrix $\theta$ to store these parameters
- $P(w|z)$: the distribution of words for a given topic ("word distribution for topic $z$"); we have $K M$ such parameters; we use matrix $\beta$ to store these parameters

The model is given by:

- $P(d,w) = P(d) P(w|d) = P(d) \sum_{k} P(z=k|d) P(w|z=k)$

Throughout this document we assume that $P(d)$ is uniform and so we can simplify our formulation:

- $P(d,w) \propto \sum_{k} P(z=k|d) P(w|z=k)$


## 2. Data generation and toy example

I will be using a toy example to illustrate the definitions and computations. The toy example assumes *three* topics ("uppercase", "lowercase", "number"), and the vocabulary is going to be composed of *six* words: $\{A,B,a,b,1,2\}$. The toy example is chosen so that the parameter values have a clear meaning. Of course, in a real corpus we will not expect such "clean" topics.

- word distributions for topics are ($\beta$):

|       |A    |B|a|b|1|2|
|-------|-----|--|---|-----|----|-----|
uppercase |0.45|0.45|0.025|0.025|0.025|0.025|
lowercase |0..025|0.025|0.6|0.3|0.025|0.025|
number    |0.025|0.025|0.025|0.025|0.2|0.7|

- topic proportions for documents are ($\theta$):

||uppercase|lowercase|number|
|-|---------|---------|------|
1| 0.8 |0.1|0.1|
2|0.1|0.8|0.1|
3|0.1|0.1|0.8|
4|1/3|1/3|1/3|

In [ ]:
# word distributions for topics: P(w|z = k)

words = "ABab12"   # vocabulary to get actual "word" from index


beta = np.array([[0.45, 0.45, 0.025, 0.025, 0.025, 0.025],
                 [0.025, 0.025, 0.6, 0.3, 0.025, 0.025],
                 [0.025, 0.025, 0.025, 0.025, 0.2, 0.7]])

print(pd.DataFrame(data=beta, columns = [w for w in words], index=['uppercase', 'lowercase', 'number']))

In [ ]:
# topic for documents: P(z = k | d)

theta = np.array([[0.8,0.1,0.1],
                  [0.1,0.8,0.1],
                  [0.1,0.1,0.8],
                  [1/3,1/3,1/3]])

print(pd.DataFrame(data=theta, index = range(1, 5), columns = ['uppercase', 'lowercase', 'number']))

### Generative process: sample "fake" corpus from given model

Using the generative mechanism of the model we could sample a possible corpus from this particular model (we assume that each document is 20 "words" long):

In [ ]:
corpus = []

doc_length = 100

K = theta.shape[0]   # number of topics, in this example K=3

np.random.seed(123)

for d in range(theta.shape[0]):         # 4 documents
    doc = []
    for i in range(doc_length):    # for each word position in doc 'd'
        
        # sample topic according to a multinomial with parameters given by theta[d, :]
        z = np.argmax(np.random.multinomial(1, theta[d,:]))  # z = 0,1,2 with probabilites given by theta[d,:]
        #print(f'chosen topic for document {d} is: {z}')

        # sample word according to topic z sampled in previous step
        w = np.argmax(np.random.multinomial(1, beta[z,:]))
        #print(f'chosen word from topic {z} for document {d} is: {w} meaning {words[w]}')
        
        # append generated word to current document
        doc.append(words[w])
        
    # append generated document to corpus
    corpus.append(doc)

# dump corpus..
for doc in corpus:
    print(' '.join(doc))

#### Compute frequency table for sampled corpus

In [ ]:
freq = [[doc.count(w) for w in words] for doc in corpus]  # frequencies

# to pretty print
print(pd.DataFrame(data = freq, columns = [w for w in words], index = range(1, theta.shape[0] + 1)))

## 3. EM algorithm for learning the parameters..

In [ ]:
N = theta.shape[0]    # 4 documents
M = beta.shape[1]     # 6 words
K = theta.shape[1]    # 3 topics

print(f'{N} documents, {M} words, {K} topics')

### set up counts

In [ ]:
# frequency counts are stored in n[d,w] array
n = np.array(freq) # make numpy array from list of lists..
print("frequency couts:", n)

# useful to compute total counts for document..
nd = n.sum(axis = 1)
print("length of docs:", nd)

### initialize parameters randomly

In [ ]:
def init_rand():
    # theta[d, k] : p(z=k|d)
    _theta = np.random.rand(N, K)
    d_sums = _theta.sum(axis = 1)
    _theta /= d_sums[:, np.newaxis]     # normalize appropriately

    # beta[k, w] : p(w|z=k)
    _beta = np.random.rand(K, M)
    w_sums = _beta.sum(axis = 1)
    _beta /= w_sums[:, np.newaxis]      # normalize appropriately

    return _theta, _beta

### E-step

The E-step computes the posterior distribution $P(z=k|d, w)$ for all $d,w$ in the corpus, given current values of $\theta$ and $\beta$
matrices. It uses the formula

$$
\begin{aligned}
P(z=k|d,w) &= \frac{P(z=k,d,w)}{P(d,w)} \\
  &\propto P(z=k,d,w) \\
  &\propto  \theta_{d,k}\ \beta_{k,w}
\end{aligned}
$$

In the code below, we can observe how the `_post` array is filled out using this formula, and afterwords the resulting result is normalized to obtain a proper probability distribution (the normalization is accross the first dimension)

In [ ]:
def EStep(theta, beta):
    
    # _post[k,d,w] : p(z=k|d,w) posterior of latent z!    
    _post = np.zeros((K,N,M))
    
    for d in range(N):
        for w in range(M):
            for k in range(K):
                _post[k, d, w] =  theta[d, k] * beta[k, w]

    # now, we normalize using first axis..
    k_sums = _post.sum(axis = 0)
    _post /= k_sums[np.newaxis,:,:]

    return _post

### M-step

The M-step updates the parameter values in the matrices $\theta$ and $\beta$ based on the updated latent variable posteriors. It does so
by invoking maximum likelihood:

$$
\begin{aligned}
\beta_{k,w} = P(w|z=k)  &=  \frac{\sum_d n(d,w) P(z=k|d,w)}{\sum_{w'} \sum_d n(d,w') P(z=k|d,w')}\\
\theta_{d,k} = P(z=k|d) &= \frac{\sum_w n(d,w) P(z=k|d,w)}{\sum_{k'} \sum_w n(d,w) P(z=k'|d,w)}
    = \frac{\sum_w n(d,w) P(z=k|d,w)}{\sum_w n(d,w)} = \frac{\sum_w n(d,w) P(z=k|d,w)}{n(d)}\\
\end{aligned}
$$

In [ ]:
def MStep(post):
    
    # re-estimate theta
    _theta = np.zeros([N, K])
    for k in range(K):
        for d in range(N):
            for w in range(M):
                _theta[d,k] += n[d,w] * post[k,d,w]
            _theta[d,k] /= nd[d]

    # re-estimate beta
    _beta = np.zeros([K, M])
    for k in range(K):
        for d in range(N):
            for w in range(M):
                _beta[k,w] += n[d,w] * post[k,d,w]
    # now normalize by axis = 1
    w_sums = _beta.sum(axis = 1)
    _beta /= w_sums[:, np.newaxis]
    
    return _theta, _beta

### Log-likelihood

$$
\begin{aligned}
   {\cal L}     &=  \sum_{d\in{\cal D}} \sum_{w\in {\cal W}} n(d,w) \log \sum_k{ P(z=k|d) P(w|z=k)}\\
        &= \sum_{d\in{\cal D}} \sum_{w\in {\cal W}} n(d,w) \log \sum_k{ \theta_{d,k}\ \beta_{k,w}}\\
\end{aligned}
$$

In [ ]:
def LogLikelihood(theta, beta):
    tmp = 0
    for d in range(N):
        for w in range(M):
            tmp += n[d,w] * np.log(theta[d,:].dot(beta[:,w]))
    return tmp

### EM algorithm

Finally, we have all elements to implement EM for pLSI. As stated in the lecture notes:

1. Randomly initialize $P(w|z)$ (matrix $\beta$) and $P(z|d)$ (matrix $\theta$)
2. Iterate until convergence:
    - E-step: _// re-compute posterior for latent variable_ $z$
        - for all $k$, compute: $P(z=k|d,w) = \frac{\theta_{d,k}\ \beta_{k,w}}{\sum_{k'}\theta_{d,k'}\ \beta_{k',w}}$
    - M-step: _// re-estimate parameters based on new posterior _
        - for all $k$, compute $\beta_{k,w} =  \frac{\sum_d n(d,w) P(z=k|d,w)}{\sum_{w'}\sum_d n(d,w') P(z=k|d,w')}$
        - for all $k$, compute $\theta_{d,k} = \frac{\sum_w n(d,w) P(z=k|d,w)}{n(d)}$


In [ ]:
import time

em_theta, em_beta = init_rand()
logl = LogLikelihood(em_theta, em_beta)
print(f'log-likelihood: {logl:.2f}')
start = time.time()
it = 0
for _ in range(100):
    t = time.time()
    z_post = EStep(em_theta, em_beta)
    em_theta, em_beta = MStep(z_post)
    elapsed = time.time() - t
    
#    print(f"iteration {i} took {elapsed:.2f} s.")
    old_logl = logl
    logl = LogLikelihood(em_theta, em_beta)
    print(f'log-likelihood: {logl:.2f}')
    
    # break of converged..
    it += 1
    if np.isclose(logl, old_logl):
        break
    
print(f"EM took {time.time()-start:.2f} s. and converged in {it} iterations.")


## 4. Inspect learned model

In [ ]:
# print word distributions for each topic, compare to true dist.
print(f'Learned word distributions:')
print(pd.DataFrame(data=em_beta, columns = [w for w in words]))
print()
print(f'True word distributions:')
print(pd.DataFrame(data=beta, columns = [w for w in words], index=['uppercase', 'lowercase', 'number']))

In [ ]:
# print topic proportions for each document, compare to true dist.
print(f'Learned topic proportions:')
print(pd.DataFrame(data = em_theta, index=range(1,5)))
print()
print(f'True topic proportions:')
print(pd.DataFrame(data = theta, index=range(1,5), columns=['uppercase', 'lowercase', 'number']))